# ADS-508-01-SP23 Team 8: Final Project

# Train model

Much of the code is modified from `Fregly, C., & Barth, A. (2021). Data science on AWS: Implementing end-to-end, continuous AI and machine learning pipelines. O’Reilly.`

## Install missing dependencies

[PyAthena](https://pypi.org/project/PyAthena/) is a Python DB API 2.0 (PEP 249) compliant client for Amazon Athena.

In [2]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0
!pip install --disable-pip-version-check -q sagemaker-experiments==0.1.26
!pip install missingno
!pip install scikit-optimize

## Globally import libraries

In [3]:
import boto3
from botocore.client import ClientError
import pandas as pd
import numpy as np
from pyathena import connect
from IPython.core.display import display, HTML
import missingno as msno
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import Lasso
import datetime as dt
import time
import sagemaker
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
import joblib
import os
from io import BytesIO

## Instantiate AWS SageMaker and S3 sessions

In [4]:
session = boto3.session.Session()
role = sagemaker.get_execution_role()
region = session.region_name
sagemaker_session = sagemaker.Session()
def_bucket = sagemaker_session.default_bucket()
bucket = 'sagemaker-us-east-ads508-sp23-t8'

s3 = boto3.Session().client(service_name="s3",
                            region_name=region)

sm = boto3.Session().client(service_name="sagemaker",
                            region_name=region)

In [5]:
setup_s3_bucket_passed = False
ingest_create_athena_db_passed = False
ingest_create_athena_table_tsv_passed = False

In [6]:
print(f"Default bucket: {def_bucket}")
print(f"Public T8 bucket: {bucket}")

Default bucket: sagemaker-us-east-1-657724983756
Public T8 bucket: sagemaker-us-east-ads508-sp23-t8


## Verify S3 Bucket Creation

In [7]:
%%bash

aws s3 ls s3://${bucket}/

2023-03-16 17:05:02 aws-athena-query-results-657724983756-us-east-1
2023-03-02 16:56:48 sagemaker-studio-657724983756-5nh7ydsouq7
2023-03-02 17:25:41 sagemaker-studio-657724983756-7yc8bp8xk0b
2023-03-02 17:01:51 sagemaker-us-east-1-657724983756
2023-03-17 05:19:31 sagemaker-us-east-ads508-sp23-t8


In [8]:
response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print(f"[ERROR] Cannot find bucket {bucket} in {response} due to {e}.")

{'ResponseMetadata': {'RequestId': 'YZJ6T6B08HP4E6VR', 'HostId': 'M0/w6z8i4PZVn97Klc2Jwi11Y0W29lFv/K0QNjbLdJkdANuGX91IhMMtzdSSC13r4263k+V6WwA=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'M0/w6z8i4PZVn97Klc2Jwi11Y0W29lFv/K0QNjbLdJkdANuGX91IhMMtzdSSC13r4263k+V6WwA=', 'x-amz-request-id': 'YZJ6T6B08HP4E6VR', 'date': 'Sat, 01 Apr 2023 18:13:38 GMT', 'x-amz-bucket-region': 'us-east-1', 'x-amz-access-point-alias': 'false', 'content-type': 'application/xml', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}


In [9]:
%store setup_s3_bucket_passed

Stored 'setup_s3_bucket_passed' (bool)


## Pass in train and test X from CSV

In [10]:
s3_train_x01_csv_path = f"s3://{def_bucket}/team_8_data/modeling_data/train_x01.csv"
train_x01 = pd.read_csv(s3_train_x01_csv_path)
s3_test_x01_csv_path = f"s3://{def_bucket}/team_8_data/modeling_data/test_x01.csv"
test_x01 = pd.read_csv(s3_test_x01_csv_path)

print(f'{train_x01.shape}')
print(f'\n{test_x01.shape}')

(25284, 48)

(6321, 48)


## Pass in train and test y from np array

In [11]:
# Define the S3 object key
train_y01_s3_key = 'team_8_data/modeling_data/train_y01.npy'

# Load the numpy array from S3
with BytesIO() as data:
    s3.download_fileobj(def_bucket, train_y01_s3_key, data)
    data.seek(0)
    train_y01 = np.load(data)

# Define the S3 object key
test_y01_s3_key = 'team_8_data/modeling_data/test_y01.npy'

# Load the numpy array from S3
with BytesIO() as data:
    s3.download_fileobj(def_bucket, test_y01_s3_key, data)
    data.seek(0)
    test_y01 = np.load(data)

# Confirm that the numpy array was loaded from S3
print(f'{train_y01.shape}')
print(f'{test_y01.shape}')

(25284, 1)
(6321, 1)


y01 = ['childpoverty']
abt_encoded_y01_vc01 = abt_encoded_df01[y01].to_numpy()
print(abt_encoded_y01_vc01.shape)
display(abt_encoded_y01_vc01[0:11])
abt_encoded_x01_df01 = abt_encoded_df01.drop(y01, axis=1)
print(abt_encoded_x01_df01.shape)
display(abt_encoded_x01_df01.head(11))

abt_encoded_x01_df01['boroughs'] = abt_encoded_x01_df01['borough_bronx'].astype(int).astype(str) + abt_encoded_x01_df01['borough_brooklyn'].astype(int).astype(str) + abt_encoded_x01_df01['borough_manhattan'].astype(int).astype(str) + abt_encoded_x01_df01['borough_queens'].astype(int).astype(str) + abt_encoded_x01_df01['borough_staten island'].astype(int).astype(str)
display(abt_encoded_x01_df01.head(5))
train_x01, test_x01, train_y01, test_y01 = train_test_split(abt_encoded_x01_df01,
                                                            abt_encoded_y01_vc01,
                                                            test_size=.2,
                                                            stratify=abt_encoded_x01_df01[['boroughs']],
                                                            shuffle=True,
                                                            random_state=1699)

train_x01 = train_x01.drop(['boroughs', 'poverty'], axis=1)
test_x01 = test_x01.drop(['boroughs', 'poverty'], axis=1)

train_y01 = train_y01.ravel()
test_y01 = test_y01.ravel()


print(f'{train_x01.shape}')
print(f'{train_y01.shape}')
print(f'\n{test_x01.shape}')
print(f'{test_y01.shape}')

## Model Training using Grid search with 5-fold cross-validation

### Random Forests

# Start timer script
start_time = dt.datetime.today()

# Citation: Hochberg, 2018; Shanmukh, 2021
m1v1_pip = Pipeline([('si', SimpleImputer(strategy='median')),
                     ('ss', StandardScaler()),
                     ('rf', RandomForestRegressor(n_jobs=2, random_state=1699))])

n_est_hparam = [1, 10, 100, 200]
max_depth_hparam = [5, 10, None]
min_samp_leaf_hparam = [1, 75, 105]
max_feat_hparam = ['sqrt', 'log2']

#n_est_hparam = [1, 10]
#max_depth_hparam = [5, 10]
#min_samp_leaf_hparam = [1, 75]
#max_feat_hparam = ['sqrt']

m1v1_grd = {'rf__n_estimators': n_est_hparam,
            'rf__max_depth': max_depth_hparam,
            'rf__min_samples_leaf': min_samp_leaf_hparam,
            'rf__max_features': max_feat_hparam
           }

m1v1_rf = GridSearchCV(m1v1_pip,
                       m1v1_grd,
                       scoring='neg_root_mean_squared_error',
                       n_jobs=2,
                       refit=True,
                       verbose=2)

m1v1_rf.fit(train_x01, train_y01)

print(f'Best Estimator:\n{m1v1_rf.best_estimator_}')

print(pd.DataFrame(m1v1_rf.cv_results_))

train_m1v1_y01_pred = m1v1_rf.predict(train_x01)
print(train_m1v1_y01_pred)

test_m1v1_y01_pred = m1v1_rf.predict(test_x01)
print(test_m1v1_y01_pred)

# Display evaluation metrics
# R-sq
train_m1v1_rf_r2 = r2_score(train_y01, train_m1v1_y01_pred)
test_m1v1_rf_r2 = r2_score(test_y01, test_m1v1_y01_pred)

print(f'Train R-sq:\n{train_m1v1_rf_r2}')
print(f'Test R-sq:\n{test_m1v1_rf_r2}')

# RMSE
train_m1v1_rf_rmse = mean_squared_error(train_y01, train_m1v1_y01_pred, squared=False)
test_m1v1_rf_rmse = mean_squared_error(test_y01, test_m1v1_y01_pred, squared=False)

print(f'Train RMSE:\n{train_m1v1_rf_rmse}')
print(f'Test RMSE:\n{test_m1v1_rf_rmse}')

# End timer script
end_time = dt.datetime.today()
time_elapse = end_time - start_time
print(f'End Time = {end_time}')
print(f'Script Time = {time_elapse}')

s3_m1v1_rf_pqt_base_path = f"../models"

if not os.path.exists(s3_m1v1_rf_pqt_base_path):
    os.makedirs(s3_m1v1_rf_pqt_base_path)

s3_m1v1_rf_pqt_path = os.path.join(s3_m1v1_rf_pqt_base_path,
                                   'm1v1_rf.parquet')

# save the model to disk using joblib
joblib.dump(m1v1_rf,
            s3_m1v1_rf_pqt_path)

# load the saved model from disk using joblib
m1v1_rf_fitted = joblib.load(s3_m1v1_rf_pqt_path)

### Neural Network

# Start timer script
start_time = dt.datetime.today()

# Citation: Hochberg, 2018; Shanmukh, 2021
m2v1_pip = Pipeline([('si', SimpleImputer(strategy='median')),
                     ('ss', StandardScaler()),
                     ('nn', MLPRegressor(random_state=1699))])

nodes_h = 3
predictors_p = 49

hidden_layer_sizes_hparam = [[100,],
                             [(nodes_h*(predictors_p+1))+nodes_h+1,],
                             [50, 50]
                            ]
activation_hparam = ['logistic', 'relu']
solver_hparam = ['adam']
alpha_hparam = [.0001, .0005, .001]
learn_rate_hparam = ['constant', 'invscaling']

#hidden_layer_sizes_hparam = [[100,]]
#activation_hparam = ['relu']
#solver_hparam = ['adam']
#alpha_hparam = [.0001]
#learn_rate_hparam = ['invscaling']

m2v1_grd = {'nn__hidden_layer_sizes': hidden_layer_sizes_hparam,
            'nn__activation': activation_hparam,
            'nn__solver': solver_hparam,
            'nn__alpha': alpha_hparam,
            'nn__learning_rate': learn_rate_hparam
           }

m2v1_nn = GridSearchCV(m2v1_pip,
                       m2v1_grd,
                       scoring='neg_root_mean_squared_error',
                       n_jobs=2,
                       refit=True,
                       verbose=2)

m2v1_nn.fit(train_x01, train_y01)

print(f'Best Estimator:\n{m2v1_nn.best_estimator_}')

print(pd.DataFrame(m2v1_nn.cv_results_))

train_m2v1_y01_pred = m2v1_nn.predict(train_x01)
print(train_m2v1_y01_pred)

test_m2v1_y01_pred = m2v1_nn.predict(test_x01)
print(test_m2v1_y01_pred)

# Display evaluation metrics
# R-sq
train_m2v1_nn_r2 = r2_score(train_y01, train_m2v1_y01_pred)
test_m2v1_nn_r2 = r2_score(test_y01, test_m2v1_y01_pred)

print(f'Train R-sq:\n{train_m2v1_nn_r2}')
print(f'Test R-sq:\n{test_m2v1_nn_r2}')

# RMSE
train_m2v1_nn_rmse = mean_squared_error(train_y01, train_m2v1_y01_pred, squared=False)
test_m2v1_nn_rmse = mean_squared_error(test_y01, test_m2v1_y01_pred, squared=False)

print(f'Train RMSE:\n{train_m2v1_nn_rmse}')
print(f'Test RMSE:\n{test_m2v1_nn_rmse}')

# End timer script
end_time = dt.datetime.today()
time_elapse = end_time - start_time
print(f'End Time = {end_time}')
print(f'Script Time = {time_elapse}')

s3_m2v1_nn_pqt_base_path = f"../models"

if not os.path.exists(s3_m2v1_nn_pqt_base_path):
    os.makedirs(s3_m2v1_nn_pqt_base_path)

s3_m2v1_nn_pqt_path = os.path.join(s3_m2v1_nn_pqt_base_path,
                                   'm2v1_nn.parquet')

# save the model to disk using joblib
joblib.dump(m2v1_nn,
            s3_m2v1_nn_pqt_path)

# load the saved model from disk using joblib
m2v1_nn_fitted = joblib.load(s3_m2v1_nn_pqt_path)

### Lasso - Using `GridSearchCV`

In [12]:
# Start timer script
start_time = dt.datetime.today()

# Citation: Hochberg, 2018; Shanmukh, 2021
m3v1_ls_pip = Pipeline([('si', SimpleImputer(strategy='median')),
                        ('ss', StandardScaler()),
                        ('ls', Lasso(random_state=1699))])

alpha_hparam = [.01, .05, .1, .5, 1, 2]
selection_hparam = ['cyclic', 'random']


m3v1_ls_grd = {'ls__alpha': alpha_hparam,
               'ls__selection': selection_hparam
           }

m3v1_ls = GridSearchCV(m3v1_ls_pip,
                       m3v1_ls_grd,
                       scoring='neg_root_mean_squared_error',
                       n_jobs=2,
                       refit=True,
                       verbose=2)

m3v1_ls.fit(train_x01, train_y01)

print(f'Best Estimator:\n{m3v1_ls.best_estimator_}')
print(f'Coefficients:\n{m3v1_ls.best_estimator_.named_steps["ls"].coef_}')

print(pd.DataFrame(m3v1_ls.cv_results_))

train_m3v1_ls_y01_pred = m3v1_ls.predict(train_x01)
print(train_m3v1_ls_y01_pred)

test_m3v1_ls_y01_pred = m3v1_ls.predict(test_x01)
print(test_m3v1_ls_y01_pred)

# Display evaluation metrics
# R-sq
train_m3v1_ls_r2 = r2_score(train_y01, train_m3v1_ls_y01_pred)
test_m3v1_ls_r2 = r2_score(test_y01, test_m3v1_ls_y01_pred)

print(f'Train R-sq:\n{train_m3v1_ls_r2}')
print(f'Test R-sq:\n{test_m3v1_ls_r2}')

# RMSE
train_m3v1_ls_rmse = mean_squared_error(train_y01, train_m3v1_ls_y01_pred, squared=False)
test_m3v1_ls_rmse = mean_squared_error(test_y01, test_m3v1_ls_y01_pred, squared=False)

print(f'Train RMSE:\n{train_m3v1_ls_rmse}')
print(f'Test RMSE:\n{test_m3v1_ls_rmse}')

# End timer script
end_time = dt.datetime.today()
time_elapse = end_time - start_time
print(f'End Time = {end_time}')
print(f'Script Time = {time_elapse}')

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:   17.0s
[Parallel(n_jobs=2)]: Done  60 out of  60 | elapsed:   20.4s finished


Best Estimator:
Pipeline(memory=None,
         steps=[('si',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='median',
                               verbose=0)),
                ('ss',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('ls',
                 Lasso(alpha=0.01, copy_X=True, fit_intercept=True,
                       max_iter=1000, normalize=False, positive=False,
                       precompute=False, random_state=1699, selection='random',
                       tol=0.0001, warm_start=False))],
         verbose=False)
Coefficients:
[-0.04577328  1.16271284  0.         -1.02188349  1.13824356 -0.
 -0.          0.          0.02926359 -0.         -0.02410066 -0.
  0.01199356 -0.         -0.          0.         -0.          2.92818225
  2.50144703  5.6083571   1.40163864 -2.3430507  -1.18949503 -0.33823435
 -1.36576657 -3.47372848 -6.793463

In [13]:
coef_intercept = np.hstack((m3v1_ls.best_estimator_.named_steps["ls"].coef_,
                            m3v1_ls.best_estimator_.named_steps["ls"].intercept_))
#print(coef_intercept)

coef_intercept_df01 = pd.DataFrame(coef_intercept)
#display(coef_intercept_df01)

train_x01_col_names = list(train_x01.columns)
train_x01_col_names.append('intercept')

train_x01_col_names_df01 = pd.DataFrame(train_x01_col_names)
#display(train_x01_col_names_df01)

model_params = pd.concat([train_x01_col_names_df01, coef_intercept_df01], axis=1)
display(model_params)

,0,0
0,borough_bronx,-0.045773
1,borough_brooklyn,1.162713
2,borough_manhattan,0.000000
3,borough_queens,-1.021883
4,borough_staten island,1.138244
5,relative_data_year_-4,-0.000000
6,relative_data_year_-3,-0.000000
7,relative_data_year_-2,0.000000
8,relative_data_year_-1,0.029264
9,relative_data_year_0,-0.000000


### Lasso - Using `BayesSearchCV`

In [14]:
# Start timer script
start_time = dt.datetime.today()

# Citation: Hochberg, 2018; Shanmukh, 2021
m3v2_ls_pip = Pipeline([('si', SimpleImputer(strategy='median')),
                        ('ss', StandardScaler()),
                        ('ls', Lasso(random_state=1699))])

alpha_hparam = Real(1e-3, 1e3, prior='log-uniform')
selection_hparam = Categorical(['cyclic', 'random'])
max_iter_hparam = Integer(100, 5000, prior='log-uniform')
warm_start_hparam = Categorical([False, True])


m3v2_ls_grd = {'ls__alpha': alpha_hparam,
               'ls__selection': selection_hparam,
               'ls__max_iter': max_iter_hparam,
               'ls__warm_start': warm_start_hparam
           }

m3v2_ls = BayesSearchCV(m3v2_ls_pip,
                       m3v2_ls_grd,
                       scoring='neg_root_mean_squared_error',
                        cv=5,
                       n_jobs=2,
                       refit=True,
                       verbose=2)

m3v2_ls.fit(train_x01, train_y01)

print(f'Best Estimator:\n{m3v2_ls.best_estimator_}')
print(f'Coefficients:\n{m3v2_ls.best_estimator_.named_steps["ls"].coef_}')

print(pd.DataFrame(m3v2_ls.cv_results_))

train_m3v2_ls_y01_pred = m3v2_ls.predict(train_x01)
print(train_m3v2_ls_y01_pred)

test_m3v2_ls_y01_pred = m3v2_ls.predict(test_x01)
print(test_m3v2_ls_y01_pred)

# Display evaluation metrics
# R-sq
train_m3v2_ls_r2 = r2_score(train_y01, train_m3v2_ls_y01_pred)
test_m3v2_ls_r2 = r2_score(test_y01, test_m3v2_ls_y01_pred)

print(f'Train R-sq:\n{train_m3v2_ls_r2}')
print(f'Test R-sq:\n{test_m3v2_ls_r2}')

# RMSE
train_m3v2_ls_rmse = mean_squared_error(train_y01, train_m3v2_ls_y01_pred, squared=False)
test_m3v2_ls_rmse = mean_squared_error(test_y01, test_m3v2_ls_y01_pred, squared=False)

print(f'Train RMSE:\n{train_m3v2_ls_rmse}')
print(f'Test RMSE:\n{test_m3v2_ls_rmse}')

# End timer script
end_time = dt.datetime.today()
time_elapse = end_time - start_time
print(f'End Time = {end_time}')
print(f'Script Time = {time_elapse}')

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    1.6s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    1.6s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    1.3s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    5.1s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    5.1s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    1.1s finished


Best Estimator:
Pipeline(memory=None,
         steps=[('si',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='median',
                               verbose=0)),
                ('ss',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('ls',
                 Lasso(alpha=0.002188020033288823, copy_X=True,
                       fit_intercept=True, max_iter=1006, normalize=False,
                       positive=False, precompute=False, random_state=1699,
                       selection='cyclic', tol=0.0001, warm_start=True))],
         verbose=False)
Coefficients:
[-5.82922236e-02  1.20418285e+00  0.00000000e+00 -1.03494899e+00
  1.14199963e+00 -0.00000000e+00 -0.00000000e+00  4.60525109e-03
  3.73130083e-02 -0.00000000e+00 -3.80975614e-02  1.70700666e-03
  0.00000000e+00 -0.00000000e+00 -2.26438004e-02  0.00000000e+00
 -0.00000000e+00  8.51853402e+00  0

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1445.3489556270652, tolerance: 903.328496061699
  positive)


In [15]:
coef_intercept = np.hstack((m3v2_ls.best_estimator_.named_steps["ls"].coef_,
                            m3v2_ls.best_estimator_.named_steps["ls"].intercept_))
#print(coef_intercept)

coef_intercept_df01 = pd.DataFrame(coef_intercept)
#display(coef_intercept_df01)

train_x01_col_names = list(train_x01.columns)
train_x01_col_names.append('intercept')

train_x01_col_names_df01 = pd.DataFrame(train_x01_col_names)
#display(train_x01_col_names_df01)

model_params = pd.concat([train_x01_col_names_df01, coef_intercept_df01], axis=1)
display(model_params)

,0,0
0,borough_bronx,-0.058292
1,borough_brooklyn,1.204183
2,borough_manhattan,0.000000
3,borough_queens,-1.034949
4,borough_staten island,1.142000
5,relative_data_year_-4,-0.000000
6,relative_data_year_-3,-0.000000
7,relative_data_year_-2,0.004605
8,relative_data_year_-1,0.037313
9,relative_data_year_0,-0.000000


## Release Resources

In [16]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [17]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>